In [78]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
import matplotlib.pyplot as plt


In [79]:
df=pd.read_csv("./data/players.csv",low_memory=False)
df.describe()

,ATPRank,games,wins,loses,players_odds,indoors_win,indoors_loss,outdoors_win,outdoors_loss,ratio_win,players_indoors_win_ratio,players_outdoors_win_ratio
count,1429.000000,1429.000000,1429.000000,1429.000000,896.000000,1429.000000,1429.000000,1429.000000,1429.000000,1429.000000,871.000000,1257.000000
mean,303.211337,72.212736,36.106368,36.106368,3.461692,6.447166,6.447166,29.659202,29.659202,0.256412,0.294805,0.266205
std,290.360729,153.371780,93.132552,66.013197,1.580138,17.592040,12.914480,77.252592,53.834708,0.219615,0.239869,0.214325
min,0.000000,1.000000,0.000000,0.000000,1.135000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,129.000000,2.000000,0.000000,1.000000,2.658006,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
50%,222.000000,6.000000,2.000000,5.000000,3.147096,0.000000,1.000000,1.000000,4.000000,0.285714,0.333333,0.307692
75%,378.000000,61.000000,22.000000,37.000000,3.749802,3.000000,6.000000,18.000000,32.000000,0.436364,0.500000,0.437500
max,3065.000000,1223.000000,1017.000000,425.000000,18.230000,167.000000,94.000000,941.000000,346.000000,1.000000,1.000000,1.000000


In [80]:
df.shape

elo 1500
gagne +32
perds -32


progression : 0.5
gagne 0.1 /anciennete

note cachée  = 1500
-1 par jour 
gangner/perdu + ancienneté match + rank atp adversaire

la note ajoute la temporalité




(1429, 13)

In [118]:
df = pd.read_csv("./data/atp_data.csv")
df.Tournament.value_counts().head(10)

Tournament
French Open                                   2794
Australian Open                               2794
US Open                                       2667
Wimbledon                                     2667
BNP Paribas Open                              1330
Sony Ericsson Open                            1235
Monte Carlo Masters                           1195
Western & Southern Financial Group Masters    1132
Internazionali BNL d'Italia                    974
Rogers Masters                                 770
Name: count, dtype: int64

In [15]:
df[df.games > 6]

,name,ATPRank,games,wins,loses,players_odds,indoors_win,indoors_loss,outdoors_win,outdoors_loss,ratio_win,players_indoors_win_ratio,players_outdoors_win_ratio
3,Acasuso J.,256,271,141,130,2.358858,8,15,133,115,0.520295,0.347826,0.536290
6,Agassi A.,39,118,85,33,1.700397,12,5,73,28,0.720339,0.705882,0.722772
12,Al Ghareeb M.,514,7,1,6,NaN,0,0,1,6,0.142857,NaN,0.142857
16,Albot R.,97,215,86,129,3.044603,26,26,60,103,0.400000,0.500000,0.368098
18,Alcaraz C.,1,164,130,34,1.773790,10,5,120,29,0.792683,0.666667,0.805369
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,di Mauro A.,135,48,14,34,3.216263,0,1,14,33,0.291667,0.000000,0.297872
1424,di Pasquale A.,228,12,3,9,2.719500,0,0,3,9,0.250000,NaN,0.250000
1425,van Gemerden M.,106,10,4,6,3.455000,0,0,4,6,0.400000,NaN,0.400000
1426,van Lottum J.,493,17,3,14,2.726000,1,1,2,13,0.176471,0.500000,0.133333


In [60]:
    
from bs4 import BeautifulSoup as bs
import requests 
atpurl = "https://www.atptour.com/en/scores/current/us-open/560/daily-schedule"
page = requests.get(atpurl)  
bs_atp = bs(page.text, "html.parser")

test = bs_atp.find('div', attrs={'class':"sectioned-day-tables"});
print(bs_atp)


<!DOCTYPE html>
<html lang="en-US"><head><title>Just a moment...</title><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="IE=Edge" http-equiv="X-UA-Compatible"/><meta content="noindex,nofollow" name="robots"/><meta content="width=device-width,initial-scale=1" name="viewport"/><link href="/cdn-cgi/styles/challenges.css" rel="stylesheet"/></head><body class="no-js"><div class="main-wrapper" role="main"><div class="main-content"><noscript><div id="challenge-error-title"><div class="h2"><span class="icon-wrapper"><div class="heading-icon warning-icon"></div></span><span id="challenge-error-text">Enable JavaScript and cookies to continue</span></div></div></noscript></div></div><script>(function(){window._cf_chl_opt={cvId: '2',cZone: 'www.atptour.com',cType: 'managed',cNounce: '38955',cRay: '7fb25e5e8ae6153d',cHash: 'aefeebfb37f26d9',cUPMDTk: "\/en\/scores\/current\/us-open\/560\/daily-schedule?__cf_chl_tk=l6OSNubSQAVxxN0YiRkBixUT1kZx1Ql0SnJNpHdF8zA-16927833

In [1]:
#obviously make sure you have chrome installed....
 
import time 
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup as bs
from selenium import webdriver 
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.common.by import By 
from webdriver_manager.chrome import ChromeDriverManager

options = ChromeOptions()
options.add_argument("--headless=new")
options.page_load_strategy = 'normal' 
chrome_path = "./bin/chromedriver.exe" #ChromeDriverManager()#.install() 
chrome_service = Service(chrome_path) 
driver = Chrome(options=options, service=chrome_service) 
driver.implicitly_wait(5)

atpurl = "https://www.atptour.com/en/scores/current/us-open/560/daily-schedule"
driver.get(atpurl) 
#time.sleep(5)
bs_atp = bs(driver.page_source, "html.parser")

In [3]:
# Getting players list 
table = bs_atp.find('div', attrs={'class':"sectioned-day-tables"}).find_all('td',attrs={'class':'day-table-name'})
players=[]
playersedited=[]
matchplayers=[]
for torototo in table:
    bala = torototo.find('a')
    players.append(bala.get('data-ga-label'))

# Transformation to fit our database name (full name, first name only first letter then .)
# example : Rodionov J.
[playersedited.append(x.split(' ')[1] +" "+ x.split(' ')[0][0] + ".") for x in players]


# Making it 2 dim
# [['Bagnis F.', 'Vandecasteele Q.'],
#  ['Bouchard E.', 'Hui K.']]
for i in range(0, len(playersedited), 2):
    matchplayers.append([playersedited[i], playersedited[i+1]])

matchplayers

[['Yastremska D.', 'Bouchard E.'],
 ['Crawley F.', 'Babos T.'],
 ['Anderson K.', 'Machac T.'],
 ['Cressy M.', 'de J.'],
 ['Couacaud E.', 'Kukushkin M.'],
 ['Bjorklund M.', 'Jeanjean L.'],
 ['Bara I.', 'Uchijima M.'],
 ['Coria F.', 'Damm M.'],
 ['Agustin T.', 'Holt B.'],
 ['Chirico L.', 'Gadecki O.'],
 ['Lys E.', 'Parrizas N.'],
 ['Siegemund L.', 'Waltert S.'],
 ['Duckworth J.', 'Atmane T.'],
 ['Hruncakova V.', 'Arango E.'],
 ['Seyboth T.', 'Kolar Z.'],
 ['Droguet T.', 'Sandgren T.'],
 ['Sramkova R.', 'Zidansek T.'],
 ['Broady L.', 'De J.'],
 ['Giannessi A.', 'Zeppieri G.'],
 ['Birrell K.', 'Semenistaja D.'],
 ['Klein L.', 'Marterer M.'],
 ['Papamichail D.', 'Zvonareva V.'],
 ['Melnikova M.', 'Dodin O.'],
 ['Hsiou Y.', 'Paire B.'],
 ['Savinykh V.', 'Lily Y.'],
 ['Minnen G.', 'Kraus S.'],
 ['Zheng M.', 'Virtanen O.'],
 ['Stefanini L.', 'Han N.'],
 ['Riera J.', 'Bektas E.'],
 ['Millman J.', 'Travaglia S.'],
 ['Mandlik E.', 'Bondar A.'],
 ['Mpetshi G.', 'Mayo A.'],
 ['Svajda Z.', 'Pospisil

In [103]:
players = pd.read_csv("./data/players.csv")

## setting up a dataframe with expected column

col_datas=[]
for col in players.columns:
    col_datas.append("player1_"+ col)  

for col in players.columns:
    col_datas.append("player2_"+ col)  


datas=pd.DataFrame(columns=col_datas)

for pp in matchplayers:
    if players[players.name==pp[0]].shape[0] == 1:
        p1 = (players[players.name==pp[0]]).values.tolist()[0]
    else:
        dfp1 = (players[players.name==pp[0]]).copy(deep=True) # copy is to avoid the settingwithcopy warning on loc
        dfp1.loc[0,"name"] = pp[0]
        p1 = dfp1.values.tolist()[0]

    if players[players.name==pp[1]].shape[0] == 1:
        p2 = players[players.name==pp[1]].values.tolist()[0]
    else:
        dfp2 = (players[players.name==pp[1]]).copy(deep=True) # copy is to avoid the settingwithcopy warning on loc
        dfp2.loc[0,"name"] = pp[1]
        p2 = dfp2.values.tolist()[0] 

    data = p1 + p2
    datas.loc[len(datas)] = data
datas#.loc[:,["player1_name","player2_name"]]

,player1_name,player2_name
0,Yastremska D.,Bouchard E.
1,Crawley F.,Babos T.
2,Anderson K.,Machac T.
3,Cressy M.,de J.
4,Couacaud E.,Kukushkin M.
5,Bjorklund M.,Jeanjean L.
6,Bara I.,Uchijima M.
7,Coria F.,Damm M.
8,Agustin T.,Holt B.
9,Chirico L.,Gadecki O.


In [17]:
print(datas)

Empty DataFrame
Columns: []
Index: []
